Deposition of integrative models 
================================



In this tutorial we will introduce the procedure used to deposit integrative modeling studies in the [PDB-Dev](https://pdb-dev.wwpdb.org/) database in mmCIF format.

We will demonstrate the procedure using [IMP](https://integrativemodeling.org/) and its PMI module, but the database will accept integrative models from any software, as long as they are compliant mmCIF files (e.g. there are several HADDOCK and Rosetta models already in PDB-Dev). 

# Why PDB-Dev?

PDB-Dev is a database run by the wwPDB. It is specifically for the deposition of *integrative* models, i.e. models generated using more than one source of experimental data. (Models that use only one experiment generally go in PDB; models that use no experimental information - theoretical models - go in [ModelArchive](https://www.modelarchive.org/)).

# Why mmCIF?

wwPDB already uses the mmCIF file format for X-ray structures, and has a formal data model ([PDBx](http://mmcif.wwpdb.org/)) to describe these structures. The format is *extensible*; extension dictionaries exist to describe NMR, SAS, EM data, etc. For integrative models, we use PDBx plus an "integrative/hybrid methods" (IHM) [extension dictionary](http://mmcif.wwpdb.org/dictionaries/mmcif_ihm.dic/Index/). This supports coarse-grained structures, multiple input experimental data sources, multiple states, multiple scales, and ensembles related by time or other order.

# Why can't we convert PDB/RMF directly to mmCIF?

This generally isn't possible because PDB, RMF and [IMP::Model](https://integrativemodeling.org/2.11.1/doc/ref/classIMP_1_1Model.html) are designed to store one or more output models, where each model is a set of coordinates for a single conformation of the system being studied. A deposition, on the other hand, aims to cover a complete **modeling study**, and should capture not just the entire ensemble of output models, but also all of the input data needed to reproduce the modeling, and quality metrics such as the precision of the ensemble and the degree to which it fits the output data. A deposition is also designed to be visualized, and so may contain additional data not used in the modeling itself, such as preset colors or views to match figures in the publication or highlight regions of interest, and more human-descriptive names for parts of the system. Thus, deposition is largely a data-gathering exercise, and benefits from a modeling study being tidy and well organized (for example, by storing it in a [GitHub](https://github.com/) repository) so that data is easy to find and track to its source.

# Generation of mmCIF files

mmCIF is a text format with a well-defined syntax, so in principle files could be generated by hand or with simple scripts. However, it is generally easier to use the existing [python-ihm library](https://github.com/ihmwg/python-ihm). This stores the same data as in an mmCIF file, but represents it as a set of Python classes, so it is easier to manipulate.

For deposition, we could use the python-ihm library directly, by writing a Python script that reads in output models and input data, adds annotations, and writes out an mmCIF file. However, since in this case we used [IMP.pmi](https://integrativemodeling.org/2.11.1/doc/ref/namespaceIMP_1_1pmi.html) to do the modeling, we can make use of a class in PMI called [ProtocolOutput](https://integrativemodeling.org/2.11.1/doc/ref/classIMP_1_1pmi_1_1mmcif_1_1ProtocolOutput.html) that automatically captures an entire [IMP.pmi](https://integrativemodeling.org/2.11.1/doc/ref/namespaceIMP_1_1pmi.html) modeling protocol.

# Basic usage of ProtocolOutput

[ProtocolOutput](https://integrativemodeling.org/2.11.1/doc/ref/classIMP_1_1pmi_1_1mmcif_1_1ProtocolOutput.html) is designed to be attached to a top-level PMI object (usually [IMP.pmi.topology.System](https://integrativemodeling.org/2.11.1/doc/ref/classIMP_1_1pmi_1_1topology_1_1System.html)). Then, as the script is run, it will capture all of the information IMP knows about the modeling study, in an [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) object. Additional information not in the modeling script itself, such as the resulting publication, can then be added using the [python-ihm API](https://python-ihm.readthedocs.io/en/latest/usage.html).

We now proceed by modifying the script from the previous tutorial to attach a ProtocolOutput object and capture modeling protocol information as mmCIF.

The first modification is to import the PMI mmCIF and python-ihm Python modules: 

In [ ]:
from __future__ import print_function

# Imports needed to use ProtocolOutput
import IMP.pmi.mmcif
import ihm

The script then proceeds as before until we have set up our top-level [IMP.pmi.topology.System](https://integrativemodeling.org/2.11.1/doc/ref/classIMP_1_1pmi_1_1topology_1_1System.html) object:

In [ ]:
import IMP
import IMP.core
import IMP.pmi.restraints.crosslinking
import IMP.pmi.restraints.stereochemistry
import IMP.pmi.restraints.em
import IMP.pmi.representation
import IMP.pmi.tools
import IMP.pmi.samplers
import RMF
import IMP.rmf

import IMP.pmi.macros
import IMP.pmi.topology

import os
import sys

import warnings
warnings.filterwarnings('ignore')


# Then setup the relevant paths of the input files

# In[2]:
try:
    import IMP.mpi
    print('ReplicaExchange: MPI was found. Using Parallel Replica Exchange')
    rex_obj = IMP.mpi.ReplicaExchange()
except ImportError:
    print('ReplicaExchange: Could not find MPI. Using Serial Replica Exchange')
    rex_obj = IMP.pmi.samplers._SerialReplicaExchange()

replica_number = rex_obj.get_my_index()


output_prefix=""
step=int(1)

datadirectory = "data/"
topology_file = datadirectory+"topology_poliii.cryoem.txt"
target_gmm_file=datadirectory+'%d_imp.gmm'%(step)
output_dir='.'
output_directory='%s/%s_output_%d/'%(output_dir,output_prefix, step)



# Initialize IMP model
m = IMP.Model()

# Read in the topology file.  
# Specify the directory wheere the PDB files, fasta files and GMM files are
topology = IMP.pmi.topology.TopologyReader(topology_file,
                                  pdb_dir=datadirectory,
                                  fasta_dir=datadirectory,
                                  gmm_dir=datadirectory)




# Use the BuildSystem macro to build states from the topology file
bs = IMP.pmi.macros.BuildSystem(m)

Now we can attach a ProtocolOutput object (BuildSystem contains a `system` member):

In [ ]:
# Record the modeling protocol to an mmCIF file
po = IMP.pmi.mmcif.ProtocolOutput(None)
bs.system.add_protocol_output(po)
po.system.title = "Modeling of RNA Pol III"
# Add publication
po.system.citations.append(ihm.Citation.from_pubmed_id(25161197))

Note that the `ProtocolOutput` object `po` simply wraps an `ihm.System` object as `po.system`. We can then customize the `ihm.System` by setting a human-readable title and adding a citation (here we use [ihm.Citation.from_pubmed_id](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.Citation.from_pubmed_id), which looks up a citation by PubMed ID - this particular PubMed ID is actually for the previously-published [modeling of the Nup84 complex](https://salilab.org/nup84/)).

Now the original script proceeds as before, setting up the representation and restraints:

In [ ]:
# Each state can be specified by a topology file.
bs.add_state(topology)




root_hier, dof = bs.execute_macro(max_rb_trans=4.0,
                                  max_rb_rot=0.3,
                                  max_bead_trans=4.0,
                                  max_srb_trans=4.0,
                                  max_srb_rot=0.3)





import IMP.pmi.plotting
import IMP.pmi.plotting.topology

IMP.pmi.plotting.topology.draw_component_composition(dof)

fixed_particles=[]
for prot in ["ABC23"]:
    fixed_particles+=IMP.atom.Selection(root_hier,molecule=prot).get_selected_particles()


# Fix the Corresponding Rigid movers and Super Rigid Body movers using dof
# The flexible beads will still be flexible (fixed_beads is an empty list)!
fixed_beads,fixed_rbs=dof.disable_movers(fixed_particles,
                                         [IMP.core.RigidBodyMover,IMP.pmi.TransformMover])




if step==1:
    # Shuffle the rigid body configuration of only the molecules we are interested in (Rpb4 and Rpb7)
    # but all flexible beads will also be shuffled.
    IMP.pmi.tools.shuffle_configuration(root_hier,
                                        max_translation=300,
                                        verbose=True,
                                        cutoff=5.0,
                                        niterations=100)
                                        #excluded_rigid_bodies=fixed_rbs,


else:
    rh_ref = RMF.open_rmf_file_read_only('seed_%d.rmf3'%(step-1))
    IMP.rmf.link_hierarchies(rh_ref, [root_hier])
    IMP.rmf.load_frame(rh_ref, RMF.FrameID(replica_number))

outputobjects = [] # reporter objects...output is included in the stat file



# Connectivity keeps things connected along the backbone (ignores if inside same rigid body)
mols = IMP.pmi.tools.get_molecules(root_hier)
for mol in mols:
    molname=mol.get_name()
    IMP.pmi.tools.display_bonds(mol)
    cr = IMP.pmi.restraints.stereochemistry.ConnectivityRestraint(mol,scale=2.0)
    cr.add_to_model()
    cr.set_label(molname)
    outputobjects.append(cr)


# #### Excluded Volume Restraint <a name="Excluded_Volume_Restraint_4"></a>

# In[12]:


ev = IMP.pmi.restraints.stereochemistry.ExcludedVolumeSphere(
                                         included_objects=root_hier,
                                         resolution=10)
ev.add_to_model()         # add to scoring function
outputobjects.append(ev)  # add to output


# #### Crosslinks - dataset 1 <a name="Crosslink_1_4"></a>
# 


# We then initialize a CrossLinkDataBase that uses a keywords converter to map column to information.
# The required fields are the protein and residue number for each side of the crosslink.
xldbkwc = IMP.pmi.io.crosslink.CrossLinkDataBaseKeywordsConverter()
xldbkwc.set_protein1_key("Protein1")
xldbkwc.set_protein2_key("Protein2")
xldbkwc.set_residue1_key("AbsPos1")
xldbkwc.set_residue2_key("AbsPos2")
xldbkwc.set_id_score_key("ld-Score")

xl1 = IMP.pmi.io.crosslink.CrossLinkDataBase(xldbkwc)
xl1.create_set_from_file(datadirectory+'FerberKosinski2016_apo.csv')
xl1.set_name("APO")

xl2 = IMP.pmi.io.crosslink.CrossLinkDataBase(xldbkwc)
xl2.create_set_from_file(datadirectory+'FerberKosinski2016_DNA.csv')
xl2.set_name("DNA")

# Append the xl2 dataset to the xl1 dataset to create a larger dataset
xl1.append_database(xl2)

# Rename one protien name
xl1.rename_proteins({"ABC14.5":"ABC14_5"})

# Create 3 confidence classes
#xl1.classify_crosslinks_by_score(3)



# Now, we set up the restraint.
xl1rest = IMP.pmi.restraints.crosslinking.CrossLinkingMassSpectrometryRestraint(
                                   root_hier=root_hier,  # The root hierarchy
                                   CrossLinkDataBase=xl1,# The XLDB defined above
                                   length=21.0,          # Length of the linker in angstroms
                                   slope=0.002,           # A linear term that biases XLed
                                                         # residues together
                                   resolution=1.0,       # Resolution at which to apply the restraint. 
                                                         # Either 1 (residue) or 0 (atomic)
                                   label="XL",         # Used to label output in the stat file
                                   weight=10.)            # Weight applied to all crosslinks 
                                                         # in this dataset
xl1rest.add_to_model()
outputobjects.append(xl1rest)

# Electron Microscopy Restraint
#  The GaussianEMRestraint uses a density overlap function to compare model to data
#   First the EM map is approximated with a Gaussian Mixture Model (done separately)
#   Second, the components of the model are represented with Gaussians (forming the model GMM)

import IMP.bayesianem
import IMP.bayesianem.restraint
# First, get the model density objects that will be fitted to the EM density.
densities = IMP.atom.Selection(root_hier, representation_type=IMP.atom.DENSITIES).get_selected_particles()
gem = IMP.bayesianem.restraint.GaussianEMRestraintWrapper(densities,
                                                 target_fn=target_gmm_file,
                                                 scale_target_to_mass=True,
                                                 slope=0.01,
                                                 target_radii_scale=3.0,
                                                 target_is_rigid_body=False)

gem.add_to_model()
gem.set_label("Total")
outputobjects.append(gem)

We can save time when it comes to the actual sampling by skipping it entirely (and using the previously-generated trajectory) by turning on [ReplicaExchange0](https://integrativemodeling.org/2.11.1/doc/ref/classIMP_1_1pmi_1_1macros_1_1ReplicaExchange0.html)'s `test_mode`:

In [ ]:
# total number of saved frames
num_frames = 2

# This object defines all components to be sampled as well as the sampling protocol
mc1=IMP.pmi.macros.ReplicaExchange0(m,
              root_hier=root_hier,                         # The root hierarchy
              monte_carlo_sample_objects=dof.get_movers()+xl1rest.get_movers(), # All moving particles and parameters
              rmf_output_objects=outputobjects,            # Objects to put into the rmf file
              crosslink_restraints=[xl1rest],      # allows XLs to be drawn in the RMF files
              monte_carlo_temperature=1.0,
              replica_exchange_minimum_temperature=1.0,
              replica_exchange_maximum_temperature=2.5,
              simulated_annealing=False,
              number_of_best_scoring_models=0,
              monte_carlo_steps=10,
              number_of_frames=num_frames,
              #save_coordinates_mode="25th_score",
              global_output_directory=output_directory,
              test_mode=True)

# Start Sampling
mc1.execute_macro()

Once we're done with our PMI protocol, we call the [ProtocolOutput.finalize](https://integrativemodeling.org/2.11.1/doc/ref/classIMP_1_1pmi_1_1mmcif_1_1ProtocolOutput.html#a7b8395a33cbd3211cdf28e943c35dfa5) method to collect all of the information about the integrative modeling protocol in [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System):

In [ ]:
po.finalize()

We can now query the [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) object, stored as the `po.system` attribute:

In [ ]:
s = po.system
print(s)
print(s.restraints)
print(len(s.entities))
print(len(s.asym_units))
print("".join(r.code for r in s.asym_units[0].entity.sequence))
print(s.restraints[0].dataset.location.path)

We can then save the entire protocol to an mmCIF file (or to BinaryCIF, if we have the Python [msgpack](https://pypi.org/project/msgpack/) package installed):

In [ ]:
import ihm.dumper
with open('rnapoliii.cif', 'w') as fh:
    ihm.dumper.write(fh, [s])

#with open('rnapoliii.bcif', 'wb') as fh:
#    ihm.dumper.write(fh, [s])

# Linking to other data

Integrative modeling draws on data from a variety of sources, so for a complete deposition all of this data needs to be available. The data is not placed directly in the mmCIF file - rather, the file contains links. These links can be:

 - an identifier in a domain-specific database, such as PDB or EMDB.
 - a DOI where the files can be obtained.
 - a path to a file on the local disk.

Database identifiers are preferable because the databases are curated by domain experts and include domain-specific information, and the files are in standard formats. ProtocolOutput will attempt to use these where possible. For example, in this case ProtocolOutput is able to read (using the [ihm.metadata module](https://python-ihm.readthedocs.io/en/latest/metadata.html#module-ihm.metadata)) the annotations of the input crystal structure used for the modeling (see below) and determine that it is stored in the PDB, so the relevant 1WCM identitifer is included in the mmCIF file (see the `_ihm_dataset_related_db_reference` table). (Similarly, the EMDB EMD-1883 identifier is used for the EM density map.)

When a file is used for the modeling which cannot be tracked back to a database, ProtocolOutput will include its path (relative to that of the mmCIF file). For example, in this case the cross-links used are stored in simple CSV files. In addition, the Python script itself is linked from the mmCIF file. Such local paths won't be available to end users, so for deposition we need to replace these paths with database IDs or DOIs (see below).

As a further example of linkage, see the links in the previously-published [modeling of the Nup84 complex](https://salilab.org/nup84/) below. The mmCIF file links to the data directly used in the modeling (cross-links, crystal structures, electron microscopy class averages, comparative models, and Python scripts) via database IDs or DOIs. Furthermore, where available links are provided from this often-processed data back to the original data, such as templates for comparative models, mass spectometry spectra for cross-links, or micrographs for class averages:

<img src="images/links.png" width="700px" title="Nup84 file linkage" />

# Annotation of input files

ProtocolOutput, using python-ihm, will look at all input files to try to extract as much metadata as possible. As described above this is used to look up database identifiers, but it can also detect other inputs, such as the templates used for comparative modeling. Thus, it is important for deposition that all input files are annotated as well as possible:

 - deposit input files in a domain-specific database where possible and use the deposited file (which typically will contain identifying headers) in the modeling repository.
 - for PDB crystal structures, do not remove the original headers, such as the `HEADER` and `TITLE` lines.
 - for MODELLER comparative models, leave in the REMARK records and make sure that any files mentioned in `REMARK 6 ALIGNMENT:` or `REMARK 6 SCRIPT:` records are available (modify the paths if necessary, for example if you moved the PDB file into a different directory from the modeling script and alignment).
 - for manually generated PDB files, such as those extracted from a published work or generated by docking or other means, add suitable `EXPDTA` and `TITLE` records to the files for ProtocolOutput to pick up. See the [python-ihm docs](https://python-ihm.readthedocs.io/en/latest/metadata.html#ihm.metadata.PDBParser.parse_file) for more information.
 - for GMM files used for the EM density restraint, keep the original MRC file around and make sure that the `# data_fn:` header in the GMM file points to it.


At the end of the modeling, we then simply write the entire study out to the mmCIF file by using ProtocolOutput's `flush` method:

In [ ]:
po.flush()